# Sensitivity kernels for transmission fibre-optic deformation sensing

The following notebook implements the calculation of sensitivity kernels for P- and S-wave speed. The underlying measurement is a cross-correlation time shift between (artificial) observed and calculated optical phase changes.

All this is based on analytical wavefield solutions for a 3D homogeneous and unbounded Earth model. To reduce the computational cost to a tractable level, sensitivity kernels are computed only in the x-y-plane (z=0).

The notebook is educational in nature. Emphasis is on easy understanding of the structure and not on computational performance.

# 0. Python packages and figure embellishment

In [ ]:
import time
import obspy
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from obspy.signal.filter import bandpass

plt.rcParams["font.family"] = "Arial"
plt.rcParams.update({'font.size': 70})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

# 1. Input

## 1.1. Wavefield modelling parameters

The first class of input parameters concerns the physical setup, including elastic properties of the medium, the wavefield source, as well as the spatio-temporal discretisation. Some care must be taken with the spatial and temporal increments, as they control both numerical accuracy and computational cost.

In [ ]:
# Medium parameters. ================================================

# P- and S-wave velocity [m/s].
vp=8000.0
vs=5000.0
# Density [kg/m^3].
rho=3000.0

# Source parameters. ================================================

# Source location [m].
xs=900.0e3
ys=0.0
zs=0.0

# Moment tensor [Nm].
M0=np.array([[1.0,1.0,0.0],
    [1.0,-0.5,0.3],
    [0.0,0.3,-0.5]])
M=4.0*np.pi*rho*(vp**3)*np.identity(3)
print(M)

# Time and space discretisation. ====================================

# Time and frequency properties [s, Hz].
tmax=150.0
dt=0.1
freqmin=0.10
freqmax=0.50

# Computational domain [m].
x=np.arange(0.0e3,1000.0e3,2.5e3)
y=np.arange(-500.0e3,500.0e3,2.5e3)
xx,yy=np.meshgrid(x,y)

## 1.2. Fibre geometry and discretisation

The second class of input parameters concerns the geometry of the fibre. Two options are currently available: 'circle' is a circular segment, and 'curly' is a sinusoidal curve with oscillations in x-direction.

In [ ]:
# Number of grid points along the fibre.
ns=200

# Choose a shape.
shape='curly'

# Input for fibre shapes. =======================================================

if shape=='circle':
    
    # Centre of the circle segment [m].
    _x_0=200.0e3
    _y_0=0.0
    
    # Starting and ending angles of the circle segment [rad].
    _phi_s=-90.0*np.pi/180.0
    _phi_e=90.0*np.pi/180.0
    
    # Radius of the circle [m].
    _R=200.0e3
    
elif shape=='curly':
    
    # Central x-coordinate.
    _x_0=200.0e3
    # Starting and ending y-coordinates.
    _y_0=-450.0e3
    _y_1=450.0e3
    # Oscillation amplitude.
    _R=50.0e3
    # Number of oscillations.
    _n=4.0

# 2. Helper functions

In order to keep the code clean, we define a couple of functions.

## 2.1. Douple-couple and single-force elastic wavefields

The wavefield excited by a double-couple source is taken in the far-field approximation (Aki & Richards, 2002. *Quantitative Seismology*). In order to save some computing time, the calculation of the P- and S-wave contributions can be switched on or off.

The wavefield excited by a vectorial single force is needed for adjoint calculations. Also here, we implement the analytical far-field approximation.

In [ ]:
# Double-couple wavefield. ==================================================

def DC(x,y,z,M,stf,isp=True,iss=True):
    """
    Wavefield for a double-couple source specified by a moment tensor.
    x, y, z: Cartesian coordinates of the point for which the wavefield is to be computed
    M: moment tensor
    stf: source time function time series
    isp: compute P-wavefield
    iss: compute S-wavefield
    """
    
    # Direction cosines and distance.
    r=np.sqrt((x-xs)**2 + (y-ys)**2 + (z-zs)**2)
    if r>0.0:
        g=np.array([(x-xs)/r, (y-ys)/r, (z-zs)/r])
    else:
        g=np.array([0.0,0.0,0.0])
        
    # Initialise P- and S-wave contributions.
    P=np.array([0.0,0.0,0.0])
    S=np.array([0.0,0.0,0.0])
    hp=0.0
    hs=0.0
        
    # Compute P-wave contribution.
    if isp:
        
        # P-wave radiation pattern.
        P=g*np.dot(g,np.dot(M,g))
        # P-wave amplitude.
        if r>0.0: P/=4.0*np.pi*rho*(vp**3)*r
        # P-wave time evolution.
        idxp=np.where(np.min(np.abs(t-r/vp)) == np.abs(t-r/vp))[0][0]
        hp=np.roll(np.diff(stf,append=0.0)/dt,idxp)
        
    # Compute S-wave contribution.
    if iss:
        
        # S-wave radiation pattern.
        S=np.dot(M,g)-g*np.dot(g,np.dot(M,g))
        # S-wave amplitude.
        if r>0.0: S/=4.0*np.pi*rho*(vs**3)*r
        # S-wave time evolution.
        idxs=np.where(np.min(np.abs(t-r/vs)) == np.abs(t-r/vs))[0][0]
        hs=np.roll(np.diff(stf,append=0.0)/dt,idxs)
    
    # Return.
    return P[0]*hp+S[0]*hs, P[1]*hp+S[1]*hs, P[2]*hp+S[2]*hs


# Single force wavefield. ===================================================

def SF(x,y,z,xad,yad,zad,F,stf,isp=True,iss=True):
    """
    Wavefield excited by a single vectorial force. This is needed for adjoint calculations.
    x,y,z: Cartesian coordinates for which the wavefield is to be computed
    xad,yad,zad: Cartesian coordinates of the adjoint source
    F: vector components of the (adjoint) source
    stf: source-time function time series
    isp: compute P-wavefield
    iss: compute S-wavefield
    """
    
    # Direction cosines and distance.
    r=np.sqrt((x-xad)**2 + (y-yad)**2 + (z-zad)**2)
    if r>0.0:
        g=np.array([(x-xad)/r, (y-yad)/r, (z-zad)/r])
    else:
        g=np.array([0.0,0.0,0.0])
        
    # Initialise P- and S-wave contributions.
    P=np.array([0.0,0.0,0.0])
    S=np.array([0.0,0.0,0.0])
    hp=0.0
    hs=0.0
    
    # Compute P-wave contribution.
    if isp:
        
        # P-wave radiation pattern.
        P=g*np.dot(g,F)
        # P-wave amplitude.
        if r>0.0: P/=4.0*np.pi*rho*(vp**2)*r
        # P-wave time evolution.
        idxp=np.where(np.min(np.abs(t-r/vp)) == np.abs(t-r/vp))[0][0]
        hp=np.roll(stf,idxp)
        
    # Compute S-wave contribution.
    if iss:
        # S-wave radiation pattern.
        S=F-g*np.dot(g,F)
        # S-wave amplitude.
        if r>0.0: S/=4.0*np.pi*rho*(vs**2)*r
        # S-wave time evolution.
        idxs=np.where(np.min(np.abs(t-r/vs)) == np.abs(t-r/vs))[0][0]
        hs=np.roll(stf,idxs)
    
    # Return.
    return P[0]*hp+S[0]*hs, P[1]*hp+S[1]*hs, P[2]*hp+S[2]*hs
    

## 2.2. Position, tangent and normal vectors of fibre shapes

The following functions implement the position vector, the tangent unit vector, the normal vector and the arc length of the different fibre geometries. For the circular segment 'circle', this can be done analytically. For the oscillating fibre geometry 'curly', we require the approximate solution of an elliptical integral.

In [ ]:
# Define position vector. =======================================================

def position(s,shape):
    
    if shape=='circle':
    
        # Compute position.
        x=_x_0+_R*np.cos(_phi_s+s/_R)
        y=_y_0+_R*np.sin(_phi_s+s/_R)
        
    elif shape=='curly':
        
        # Compute the length of the curly line via elliptical integral.
        import scipy.special as ss
        w=2.0*np.pi*_n
        a=(_y_1-_y_0)
        m=(_R**2 * w**2)/(_R**2 * w**2 + a**2)
        L=ss.ellipk(m)
        L=L*2.0*np.sqrt(_R**2 * w**2 + a**2)/np.pi
        
        # Compute position.
        x=_x_0+_R*np.sin(w*s/L)
        y=_y_0+a*s/L

    # Return.
    return x,y


# Define tangent vector. ========================================================

def tangent(s,shape):
    
    if shape=='circle':

        # Compute normalised tangent.
        x=-np.sin(_phi_s+s/_R)
        y=np.cos(_phi_s+s/_R)
        
    elif shape=='curly':
        
        # Compute the length of the curly line via elliptical integral.
        import scipy.special as ss
        w=2.0*np.pi*_n
        a=(_y_1-_y_0)
        m=(_R**2 * w**2)/(_R**2 * w**2 + a**2)
        L=ss.ellipk(m)
        L=L*2.0*np.sqrt(_R**2 * w**2 + a**2)/np.pi
        
        # Compute non-normalised tangent.
        x=_R*w*np.cos(w*s/L)/L
        y=a/L
        
        # Normalisation factor.
        scale=np.sqrt(x**2+y**2)
        
        # Normalise.
        x=x/scale
        y=y/scale

    # Return.
    return x,y


# Define normal vector. ==========================================================

def normal(s,shape):
    
    if shape=='circle':
    
        # Compute non-normalised normal.
        x=-np.cos(_phi_s+a*s)/_R
        y=-np.sin(_phi_s+a*s)/_R
        
    elif shape=='curly':
        
        # Compute the length of the curly line via elliptical integral.
        import scipy.special as ss
        w=2.0*np.pi*_n
        a=(_y_1-_y_0)
        m=(_R**2 * w**2)/(_R**2 * w**2 + a**2)
        L=ss.ellipk(m)
        L=L*2.0*np.sqrt(_R**2 * w**2 + a**2)/np.pi
        
        # Compute non-normalised normal via finite-difference.
        ds=L/1000.0
        xa,ya=tangent(s-ds,shape)
        xb,yb=tangent(s+ds,shape)
        x=(xb-xa)/(2.0*ds)
        y=(yb-ya)/(2.0*ds)

    # Return.
    return x,y


# Length of the fibre. ===========================================================

def length(shape):
    
    if shape=='circle':
        return (_phi_e-_phi_s)*_R
    
    elif shape=='curly':
        
        # Compute the length of the curly line via elliptical integral.
        import scipy.special as ss
        w=2.0*np.pi*_n
        a=(_y_1-_y_0)
        m=(_R**2 * w**2)/(_R**2 * w**2 + a**2)
        L=ss.ellipk(m)
        L=L*2.0*np.sqrt(_R**2 * w**2 + a**2)/np.pi

        return L

# 3. Forward modelling

## 3.1. Source-time function of the forward wavefield

The first contribution to the forward model is the source time function. To mimick an earthquake source, we use a bandpass filtered version of the Heaviside function.

In [ ]:
t=np.arange(0.0,tmax,dt)
stf=np.heaviside(t,0)
stf=bandpass(stf,freqmin=freqmin,freqmax=freqmax,df=1.0/dt,corners=4,zerophase=False)

plt.subplots(1, figsize=(30,10))
plt.plot(t,stf,'k',linewidth=4)
plt.xlim(0.0,50.0)
plt.grid()
plt.savefig('./OUTPUT/stf.png',format='png',dpi='figure')
plt.show()

## 3.2. Compute wavefield snapshot for selected time step

Just for visual inspection, we compute and plot the wavefield for a certain time step and a selected component.

In [ ]:
# Select time index and wavefield component.
time_index=600
component=0

# Compute for each grid point.
ux=np.zeros(np.shape(xx))

for i in range(len(x)):
    for j in range(len(y)):
        
        ux[j,i]=DC(x[i],y[j],0.0,M,stf)[component][time_index]

In order to superimpose the fibre, we compute its geometry.

In [ ]:
# Compute geometry of the fibre.
L=length(shape)
print(L)
s=np.linspace(0.0,L,ns)
xh,yh=position(s,shape)

Then we visualise the wavefield and the fibre.

In [ ]:
# Visualise.
plt.subplots(1, figsize=(3.0*(x[-1]-x[0])/1.0e5,3.0*(y[-1]-y[0])/1.0e5))
m=1.0*np.max(np.abs(ux))
plt.pcolormesh(xx/1000.0,yy/1000.0,ux,vmin=-m,vmax=m,cmap='RdBu')
plt.plot(xh/1000.0,yh/1000.0,'k',linewidth=6)
plt.xlim(x[0]/1000.0,x[-1]/1000.0)
plt.ylim(y[0]/1000.0,y[-1]/1000.0)
plt.xlabel('x [km]',labelpad=30)
plt.ylabel('y [km]',labelpad=30)
plt.tight_layout()
plt.minorticks_on()
plt.grid(which='major',color='k',linewidth=1.0)
plt.grid(which='minor',color='k',linewidth=0.5)
plt.savefig('./OUTPUT/forward_wavefield.png',format='png',dpi='figure')
plt.show()

## 3.3. Compute optical phase shifts

Given the forward wavefield model and the fibre geometry, we can compute the measured optical time shift series. Some care must be taken here to ensure that the arc length increment is small enough.

In [ ]:
# Allocate array.
theta=np.zeros(len(t))

# Arc length increment.
ds=s[1]-s[0]

# Loop through sources.
for i in range(len(s)):
    
    xhi,yhi=position(s[i],shape)
    N=normal(s[i],shape)
    v=np.diff(DC(xhi,yhi,0.0,M,stf),append=0.0)/dt
    
    theta+=N[0]*v[0]+N[1]*v[1]
    
theta*=ds

In [ ]:
plt.subplots(1, figsize=(40,10))
plt.plot(t,theta,'k',linewidth=4)
plt.xlim(0.0,t[len(t)-1])
plt.xlabel('t [s]')
plt.minorticks_on()
plt.grid(which='major',color='k',linewidth=1.0)
plt.grid(which='minor',color='k',linewidth=0.5)
plt.tight_layout()
plt.savefig('./OUTPUT/phase_shift.png',format='png',dpi='figure')
plt.show()

# 4. Adjoint modelling

Finally, we compute sensitivity kernels. This procedure is computationally slow and may take some time.

The first step is to define a range of time windows for which we would like to compute these kernels. Then, we compute the corresponding adjoint sources, plot them, and then loop through all the discrete source points. At the end, we plot and save the kernels.

To accelerate the computation a little bit, we switch off the calculation of S-waves for the calculation of P-velocity kernels. This also allows us to use an approximate expression for the P-wave kernels that does not required the calculation of the forward and adjoint strain fields.

In [ ]:
#= Adjoint source and windowing ===================================================================================

nt=len(t)

# Make an array of start and end times of windows.
t_start=[99.2]
t_end=[103.0]

# Loop through start and end times.
for it in range(len(t_start)):
    
    # Start and end indeces of the time window.
    i_start=np.where(np.min(np.abs(t-t_start[it]))==np.abs(t-t_start[it]))[0][0]
    i_end=np.where(np.min(np.abs(t-t_end[it]))==np.abs(t-t_end[it]))[0][0]
    
    # Make adjoint source.
    adsrc=theta.copy()
    adsrc[0:i_start]=0.0
    adsrc[i_end:nt]=0.0
    for i in range(4): adsrc[1:nt-1]=(adsrc[1:nt-1]+adsrc[2:nt]+adsrc[0:nt-2])/3.0
    adsrc=np.flip(np.diff(adsrc,append=0.0)/dt)
    scale=np.sum(adsrc**2)*dt
    adsrc/=scale

    # Plot adjoint source.
    plt.subplots(1, figsize=(30,10))
    plt.plot(t,adsrc,'k')
    plt.xlim(0.0,t[len(t)-1])
    plt.minorticks_on()
    plt.grid(which='major',color='k',linewidth=1.0)
    plt.grid(which='minor',color='k',linewidth=0.5)
    plt.tight_layout()
    figurename='./OUTPUT/adsrc_'+str(it)+'.png'
    plt.savefig(figurename,format='png',dpi='figure')
    plt.show()
    
#= Compute sensitivity kernel =====================================================================================

    # Allocate adjoint field and kernels.
    Kvp=np.zeros(np.shape(xx))

    # Approximate pre-factors for kernel calculation.
    scale_p=(2.0*np.pi*freqmax/vp)**2

    # March through the source points.
    for k in range(len(s)):

        print(k)

        # Compute position and normal vectors.
        xhk,yhk=position(s[k],shape)
        n=normal(s[k],shape)
        narray=np.array([n[0],n[1],0.0])

        for i in range(len(x)):
            for j in range(len(y)):

                # Compute adjoint wavefield for that particular source point.
                uad=SF(x[i],y[j],0.0,xhk,yhk,0.0,narray,adsrc,iss=False)

                # Compute forward wavefield.
                u=DC(x[i],y[j],0.0,M,stf,iss=False)

                # Assemble kernel for that location.
                Kvp[j,i]-=np.sum(np.flip(uad[0])*np.diff(u[0],append=0.0)+np.flip(uad[1])*np.diff(u[1],append=0.0)+np.flip(uad[2])*np.diff(u[2],append=0.0))*ds

    # Scale kernels to proper dimensions for relative perturbation kernels.
    Kvp*=2.0*rho*vp*vp*scale_p
    
#= Plot and save ==================================================================================================

    # Save kernel.
    filename='./OUTPUT/Kvp_'+str(it+4)+'.npy'
    np.save(filename,Kvp)
    np.save('./OUTPUT/xx.npy',xx)
    np.save('./OUTPUT/yy.npy',yy)

    # Set up figure.
    plt.subplots(1, figsize=(3.0*(x[-1]-x[0])/1.0e5,3.0*(y[-1]-y[0])/1.0e5))

    # Scaling of the colourbar.
    m=0.07*np.max(np.abs(Kvp))

    # Make figure.
    plt.pcolormesh(xx/1000.0,yy/1000.0,Kvp,vmin=-m,vmax=m,cmap='RdBu')
    xh,yh=position(s,shape)
    plt.plot(xh/1000.0,yh/1000.0,'k',linewidth=4)
    plt.xlim(x[0]/1000.0,x[-1]/1000.0)
    plt.ylim(y[0]/1000.0,y[-1]/1000.0) 
    plt.xlabel('x [km]',labelpad=30)
    plt.ylabel('y [km]',labelpad=30)
    plt.title('K [s/m**3]',pad=30)
    plt.colorbar()
    plt.tight_layout()
    plt.grid() 
    figurename='./OUTPUT/Kvp_'+str(it)+'.png'
    plt.savefig(figurename,format='png',dpi='figure')
    plt.show()